In [ ]:
# uncommment below line to install ruptures if not already installed

# !python -m pip install ruptures

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ruptures as rpt
from scipy.stats import zscore, linregress, iqr, skew, kurtosis
import numpy as np
import xlsxwriter
import os
from datetime import datetime

# Please enter the below details

In [ ]:
penalty = 3 # higher means less cells
dim = 'Cell Name' # dimension you are interested in
date = 'Date'     # date variable
img_dir = "C:/Users/BronyahJ/images/" # path to store images
source_file_path = "C:\\Users\\BronyahJ\\Downloads\\" # where your source data is located
file_name = "Worst_Cells_5G_Query_Result_20230212113700767.xlsx" # source filename
Tech = '5G'
output_filename = 'worst_cells.xlsx'
scale_data = True
chg_thd = 0.1

# Specify columns you want to delete in the to_delete list

In [ ]:
REMOVE_NA = True
to_delete = ['LocalCell Id','Integrity', 'Cell ID', 'Cell CI', 'CellIndex','NR Cell ID']
KNOWN_NA_VALS = ['NIL', 'NILL', 'NULL', 'NA', '#NA', '#N/A', 'N/A','#VALUE!','#REF!','#DIV/0!','#NUM!','#NAME?','#NULL!','NAN','nan','NaN']

In [ ]:
imp_st = datetime.now()

if str(file_name).split(".")[-1] == 'xlsx' or str(file_name).split(".")[-1] == 'xls':
    df = pd.read_excel(source_file_path+file_name)
    
elif str(file_name).split(".")[-1] == 'csv':
    df  = pd.read_csv(source_file_path+file_name,skiprows=6)

imp_end= datetime.now()
print("It took {} time to import data".format(imp_end-imp_st))

In [ ]:
if len(to_delete) > 0:
    df = df.drop([x for x in to_delete if x in df.columns], axis=1)

# Data cleaning and processing

In [ ]:
############################################################################################
# Experimental code here. Trying to replace Known values which means NULL / NA to np.nan
# Ideally this should help our analysis.
############################################################################################
for x in [x for x in df.columns if df[x].dtype.kind.lower() in ('o', 's', 'u', 'v')]:
    if len(df[df[x].isin(KNOWN_NA_VALS)]) > 0: #some instances were found with Known NA substitutions
        df.loc[ df[x].isin(KNOWN_NA_VALS), x ] = np.nan
        print("Found some known NA substitutions in {}. Will replace and try to force as numeric".format(x))
    try: #now we will try to see if the column can become numeric
        df[x] = pd.to_numeric(df[x], errors='raise')
    except ValueError as e:
        continue #Column cannot be converted to numeric. Just continue
############################################################################################

############################################################################################
# Experiment 2: If all endings are % or $ or # then we will try to strip these and check if 
#     the column can be converted as a numeric value
############################################################################################
sp_endings = ['%', '$', '#', '£', 'QAR', 'GBP', 'qar', 'gbp', 'usd', 'USD' ,'eur', 'EUR']

for x in [x for x in df.columns if df[x].dtype.kind.lower() in ('o', 's', 'u', 'v')]:
    for sp in sp_endings:
        totals = df[x].astype(str).str.endswith(sp).sum() + df[x].isna().sum()
        if totals == len(df): #Either all entries end with special char or are null
            temp = df[x].astype(str).str.replace(sp, '')
            try:
                temp_numeric = pd.to_numeric(temp, errors='raise')
                df[x] = temp_numeric #if we were able to convert to numeric then we keep this
                                    # in our dataframe. else no change
                print("Modified column {} for special endings {} and changed to numeric".format(
                        x, sp))
            except ValueError as ve:
                continue
############################################################################################
# Same code as above but for string beginnings

for x in [x for x in df.columns if df[x].dtype.kind.lower() in ('o', 's', 'u', 'v')]:
    for sp in sp_endings:
        totals = df[x].astype(str).str.startswith(sp).sum() + df[x].isna().sum()
        if totals == len(df): #Either all entries end with special char or are null
            temp = df[x].astype(str).str.replace(sp, '')
            try:
                temp_numeric = pd.to_numeric(temp, errors='raise')
                df[x] = temp_numeric #if we were able to convert to numeric then we keep this
                                    # in our dataframe. else no change
                print("Modified column {} for special startings {} and changed to numeric".format(
                        x, sp))
            except ValueError as ve:
                continue
############################################################################################
# Now we have to deal with NA values
if REMOVE_NA == True:
    df = df.dropna(axis=0, how='all')
else:
    for x in [x for x in df.columns]:
        if df[x].dtype.kind in ('f', 'c', 'i', 'u'):
            df[x].fillna(df[x].median(),inplace=True)
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')
############################################################################################

    
############################################################################################    
# WARNING - DONT REMOVE BELOW WITHOUT UNDERSTANDING OF THE CODE BIT
# This step is mandatory. We will delete any column if it is Completely np.nan
a = df.isna().sum(axis=0)
FULL_NA_COLS = [x for x in a[df.isna().sum(axis=0) == len(df)].index]
df = df.drop(FULL_NA_COLS, axis=1)

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
df.head()

In [ ]:
df01=df.copy(deep=True)

In [ ]:
df01 = df01.dropna(axis=0, how='any')
df01.reset_index(inplace=True,drop=True)

# Worst Cell Detection and export

In [ ]:
proc_st = datetime.now()

workbook = xlsxwriter.Workbook(Tech+str(datetime.now()).split(" ")[0]+output_filename)


for x in [x for x in df.columns if df[x].dtype.kind.lower() not in ('o', 's', 'u', 'v','m')]:
    
    df1= df.pivot_table(index=dim, columns=date, values=x, aggfunc=np.mean)
    
    cells = []
    cells_2 = []
    #cells_3 = []
    net_avg = np.nanmean(df[x].values)
    df2 = df1.index.to_list()
    
    
    IQR = iqr(df[x].values, nan_policy='omit')
    Q1 = np.nanpercentile(df[x].values,25)
    Q3 = np.nanpercentile(df[x].values,75)
    low_limit = Q1 - 1.5*IQR
    high_limit = Q3 + 1.5*IQR
    cutoff_low = np.nanmean(df[x].values) - (3*np.nanstd(df[x].values))
    cutoff_high = np.nanmean(df[x].values) + (3*np.nanstd(df[x].values))
    
    sk = skew(df[x].values, nan_policy='omit')
    kurt = kurtosis(df[x].values, nan_policy='omit')
    
    high_count = 0
    low_count = 0
    for i in np.arange(0,df01[x].shape[0]):
        if (df01[x][i] >= high_limit) and (low_limit != high_limit):
            high_count = high_count + 1
            
        elif (df01[x][i] <= low_limit) and (low_limit != high_limit) :
            low_count = low_count + 1
    
    
    
    for i in np.arange(0,df1.shape[0]):
        df3 = df1.iloc[i,:]
        algo = rpt.Pelt(model="rbf").fit(df3.values)
        result = algo.predict(pen=penalty)
        if len(result)>1:
            cells.append(df2[i])
        elif len(result)==1:
            if ((sk>=0 and high_count>low_count and np.nanmean(df3.values)>cutoff_high) or (high_count==0 and low_count==0 and sk>=0 and np.nanmean(df3.values)>cutoff_high)):
                cells_2.append(df2[i])
            elif (high_count>low_count and sk<0 and kurt<=0 and np.nanmean(df3.values)>cutoff_high):
                cells_2.append(df2[i])
            elif ((sk<0 and high_count<low_count and np.nanmean(df3.values)<cutoff_low) or (high_count==0 and low_count==0 and sk<0 and np.nanmean(df3.values)<cutoff_low)):
                cells_2.append(df2[i])
            elif (high_count<low_count and sk<0 and kurt<=0 and np.nanmean(df3.values)<cutoff_low):
                cells_2.append(df2[i])
                
    
    
    
    
    
    df4= df.pivot_table(index=dim, columns=date, values=x, aggfunc=np.mean)
    
    
    if scale_data:
        for i in np.arange(0,len(df4)):
            df4.iloc[i] -= df4.iloc[i].min()
            if (df4.iloc[i].max()-df4.iloc[i].min()) > 0:
                df4.iloc[i] /= (df4.iloc[i].max()-df4.iloc[i].min())
            else:
                df4.iloc[i] = 0
                
            
            
        
    
    for i in np.arange(0,len(cells)):
        df5 = df4.loc[cells[i]]
        df6 = df1.loc[cells[i]]
        algo = rpt.Pelt(model="rbf").fit(df6.values)
        result = algo.predict(pen=penalty)
        chg_perc= (np.nanmean(df5[result[0]:result[-1]])- np.nanmean(df5[0:result[0]]))/(np.nanmean(df5[0:result[0]]) + 0.0000000000001)
        slope = linregress(range(len(df5.values)), df5.values).slope
        last_segment_avg = np.nanmean(df6[result[0]:result[-1]].values)
        if ((sk>=0 and high_count>low_count) or (high_count==0 and low_count==0 and sk>=0)):
            if abs(chg_perc)>chg_thd and slope>0 and last_segment_avg>=high_limit:
                cells_2.append(cells[i])
                
        elif (high_count>low_count and sk<0 and kurt<=0):
            if abs(chg_perc)>chg_thd and slope>0 and last_segment_avg>=high_limit:
                cells_2.append(cells[i])
                
        else:
            if abs(chg_perc)>chg_thd and slope<0 and last_segment_avg<=low_limit:
                cells_2.append(cells[i])
   

  
    
    
    worksheet = workbook.add_worksheet(name=str(x)[0:30])
    worksheet.set_column('A:A', 30)
    xl_col = 2
    
    if len(cells_2)>0:
        for i in np.arange(0,len(cells_2)):
            plt.figure(figsize = (10, 5))
            algo = rpt.Pelt(model="rbf").fit(df1.loc[cells_2[i]].values)
            result = algo.predict(pen=penalty)
            a=rpt.display(df1.loc[cells_2[i]].values, result)
            plt.title(str(df1.loc[cells_2[i]].head(0)).split(',')[1])
            plt.xticks(np.arange(0,len(df1.loc[cells_2[i]].values)),df1.loc[cells_2[i]].index,rotation='vertical')
            img_path = os.path.join(img_dir,"{}_{}_{}.png".format(str(x)[0:5],str(datetime.now()).replace(":", "_"),i))
            plt.savefig(img_path,bbox_inches="tight")
            plt.close()
            worksheet.write('A'+str(xl_col), str(df1.loc[cells_2[i]].head(0)).split(',')[1])
            worksheet.insert_image('B'+str(xl_col), img_path)
            plt.close()
            xl_col = xl_col + 20
        
    
    
workbook.close()
        
proc_end = datetime.now()

print("It took {} time to process data".format(proc_end-proc_st))

    